In [3]:
import os
import open3d as o3d
import numpy as np
from plyfile import PlyData, PlyElement
from torch import nn
import torch
from errno import EEXIST
import math

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Working on a dataset "The VR-NeRF Eyeful Tower Dataset". 

- https://github.com/facebookresearch/EyefulTower?tab=readme-ov-file
- This dataset is originally used for NeRF. 

In [26]:
'''
1. Move all the images from different cameras into the same file
2. Convert all the images to executable format for 3DGS
'''

dataset_dir = "/home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment/images-jpeg-2k"
input_dir = "/home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment/input"

In [27]:
# # for each dir in dataset_dir, move all files with extension .jpg in the dir to input_dir
# for dir in os.listdir(dataset_dir):
#     if os.path.isdir(os.path.join(dataset_dir, dir)):
#         for file in os.listdir(os.path.join(dataset_dir, dir)):
#             if file.endswith(".jpg"):
#                 os.rename(os.path.join(dataset_dir, dir, file), os.path.join(input_dir, file))
#                 print(f"move {file} to {input_dir}")

In [28]:
# uniformly select n images from the input_dir over all the images, not randomly
n = 1750
selected_files = []
input_files = os.listdir(input_dir)
input_files.sort()
# selected_files = input_files[::math.ceil((len(input_files)/n))]
interval = len(input_files) / n
index = 0
while len(selected_files) < n and index < len(input_files):
    selected_files.append(input_files[int(index)])
    index += interval


# **copy** the selected images to a new dir
output_dir = f"/home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_{n}/input"
try:
    os.makedirs(output_dir)
except OSError as e:
    if e.errno != EEXIST:
        raise
for file in selected_files:
    os.system(f"cp {os.path.join(input_dir, file)} {output_dir}")
    print(f"copy {file} to {output_dir}")


copy 10_DSC0001.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0019.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0037.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0055.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0073.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0100.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0118.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0136.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0154.jpg to /home/yuang/Desktop/3d_gaussian_splat/dataset/source/eyefultower/apartment_1750/input
copy 10_DSC0181.jpg

In [15]:
len(selected_files)

3000

In [16]:
selected_files[-100:]

['31_DSC0478.jpg',
 '31_DSC0487.jpg',
 '31_DSC0496.jpg',
 '31_DSC0523.jpg',
 '31_DSC0532.jpg',
 '31_DSC0541.jpg',
 '31_DSC0559.jpg',
 '31_DSC0568.jpg',
 '31_DSC0577.jpg',
 '31_DSC0586.jpg',
 '31_DSC0604.jpg',
 '31_DSC0613.jpg',
 '31_DSC0622.jpg',
 '31_DSC0640.jpg',
 '31_DSC0649.jpg',
 '31_DSC0658.jpg',
 '31_DSC0676.jpg',
 '31_DSC0685.jpg',
 '31_DSC0694.jpg',
 '31_DSC0712.jpg',
 '31_DSC0721.jpg',
 '31_DSC0730.jpg',
 '31_DSC0739.jpg',
 '31_DSC0757.jpg',
 '31_DSC0766.jpg',
 '31_DSC0775.jpg',
 '31_DSC0793.jpg',
 '31_DSC0802.jpg',
 '31_DSC0811.jpg',
 '31_DSC0829.jpg',
 '31_DSC0838.jpg',
 '31_DSC0847.jpg',
 '31_DSC0865.jpg',
 '31_DSC0874.jpg',
 '31_DSC0883.jpg',
 '31_DSC0892.jpg',
 '31_DSC0910.jpg',
 '31_DSC0919.jpg',
 '31_DSC0928.jpg',
 '31_DSC0946.jpg',
 '31_DSC0964.jpg',
 '31_DSC0973.jpg',
 '31_DSC0991.jpg',
 '31_DSC1000.jpg',
 '31_DSC1009.jpg',
 '31_DSC1027.jpg',
 '31_DSC1036.jpg',
 '31_DSC1045.jpg',
 '31_DSC1054.jpg',
 '31_DSC1072.jpg',
 '31_DSC1081.jpg',
 '31_DSC1090.jpg',
 '31_DSC1108

In [ ]:
# convert the images to executable format
# python convert.py -s {the parent dir of input_dir}
# os.system(f"python convert.py -s {os.path.dirname(input_dir)}")

# Using quantization to make distorted 3DGS

- Perform quantization on bit depth to make distorted 3DGS.

In [51]:
class GaussianModel:

    def __init__(self, sh_degree : int):
        self.active_sh_degree = 0
        self.max_sh_degree = sh_degree  
        self._xyz = torch.empty(0)
        self._features_dc = torch.empty(0)
        self._features_rest = torch.empty(0)
        self._scaling = torch.empty(0)
        self._rotation = torch.empty(0)
        self._opacity = torch.empty(0)
        self._normal = torch.empty(0)
        self.max_radii2D = torch.empty(0)
        self.xyz_gradient_accum = torch.empty(0)
        self.denom = torch.empty(0)
        self.optimizer = None
        self.percent_dense = 0
        self.spatial_lr_scale = 0

    def save_ply(self, path):

        xyz = self._xyz.detach().cpu().numpy()
        # normals = np.zeros_like(xyz)
        normals = self._normal.detach().cpu().numpy()
        f_dc = self._features_dc.detach().transpose(1, 2).flatten(start_dim=1).contiguous().cpu().numpy()
        f_rest = self._features_rest.detach().transpose(1, 2).flatten(start_dim=1).contiguous().cpu().numpy()
        opacities = self._opacity.detach().cpu().numpy()
        scale = self._scaling.detach().cpu().numpy()
        rotation = self._rotation.detach().cpu().numpy()

        dtype_full = [(attribute, 'f4') for attribute in self.construct_list_of_attributes()]

        elements = np.empty(xyz.shape[0], dtype=dtype_full)
        attributes = np.concatenate((xyz, normals, f_dc, f_rest, opacities, scale, rotation), axis=1)
        elements[:] = list(map(tuple, attributes))
        el = PlyElement.describe(elements, 'vertex')
        PlyData([el]).write(path)

    def load_ply(self, path):
        plydata = PlyData.read(path)

        xyz = np.stack((np.asarray(plydata.elements[0]["x"]),
                        np.asarray(plydata.elements[0]["y"]),
                        np.asarray(plydata.elements[0]["z"])),  axis=1)
        normals = np.stack((np.asarray(plydata.elements[0]["nx"]),
                            np.asarray(plydata.elements[0]["ny"]),
                            np.asarray(plydata.elements[0]["nz"])),  axis=1)
        opacities = np.asarray(plydata.elements[0]["opacity"])[..., np.newaxis]

        features_dc = np.zeros((xyz.shape[0], 3, 1))
        features_dc[:, 0, 0] = np.asarray(plydata.elements[0]["f_dc_0"])
        features_dc[:, 1, 0] = np.asarray(plydata.elements[0]["f_dc_1"])
        features_dc[:, 2, 0] = np.asarray(plydata.elements[0]["f_dc_2"])

        extra_f_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("f_rest_")]
        extra_f_names = sorted(extra_f_names, key = lambda x: int(x.split('_')[-1]))
        assert len(extra_f_names)==3*(self.max_sh_degree + 1) ** 2 - 3
        features_extra = np.zeros((xyz.shape[0], len(extra_f_names)))
        for idx, attr_name in enumerate(extra_f_names):
            features_extra[:, idx] = np.asarray(plydata.elements[0][attr_name])
        # Reshape (P,F*SH_coeffs) to (P, F, SH_coeffs except DC)
        features_extra = features_extra.reshape((features_extra.shape[0], 3, (self.max_sh_degree + 1) ** 2 - 1))

        scale_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("scale_")]
        scale_names = sorted(scale_names, key = lambda x: int(x.split('_')[-1]))
        scales = np.zeros((xyz.shape[0], len(scale_names)))
        for idx, attr_name in enumerate(scale_names):
            scales[:, idx] = np.asarray(plydata.elements[0][attr_name])

        rot_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("rot")]
        rot_names = sorted(rot_names, key = lambda x: int(x.split('_')[-1]))
        rots = np.zeros((xyz.shape[0], len(rot_names)))
        for idx, attr_name in enumerate(rot_names):
            rots[:, idx] = np.asarray(plydata.elements[0][attr_name])

        self._xyz = nn.Parameter(torch.tensor(xyz, dtype=torch.float, device="cuda").requires_grad_(True))
        self._normal = nn.Parameter(torch.tensor(normals, dtype=torch.float, device="cuda").requires_grad_(True))
        self._features_dc = nn.Parameter(torch.tensor(features_dc, dtype=torch.float, device="cuda").transpose(1, 2).contiguous().requires_grad_(True))
        self._features_rest = nn.Parameter(torch.tensor(features_extra, dtype=torch.float, device="cuda").transpose(1, 2).contiguous().requires_grad_(True))
        self._opacity = nn.Parameter(torch.tensor(opacities, dtype=torch.float, device="cuda").requires_grad_(True))
        self._scaling = nn.Parameter(torch.tensor(scales, dtype=torch.float, device="cuda").requires_grad_(True))
        self._rotation = nn.Parameter(torch.tensor(rots, dtype=torch.float, device="cuda").requires_grad_(True))

        self.active_sh_degree = self.max_sh_degree

    def construct_list_of_attributes(self):
        l = ['x', 'y', 'z', 'nx', 'ny', 'nz']
        # All channels except the 3 DC
        for i in range(self._features_dc.shape[1]*self._features_dc.shape[2]):
            l.append('f_dc_{}'.format(i))
        for i in range(self._features_rest.shape[1]*self._features_rest.shape[2]):
            l.append('f_rest_{}'.format(i))
        l.append('opacity')
        for i in range(self._scaling.shape[1]):
            l.append('scale_{}'.format(i))
        for i in range(self._rotation.shape[1]):
            l.append('rot_{}'.format(i))
        return l
    
def quantize_array(arr, bit_depth):
    # Calculate the maximum value based on the desired bit depth
    max_value = (2 ** bit_depth) - 1

    # Quantize the array
    quantized_arr = np.round(arr * max_value) / max_value

    return quantized_arr

In [55]:
# we group the features into three groups: 
    # 1. position: xyz, 
    # 2. color: normal, features_dc, features_rest, opacity
    # 3. size: scaling, rotation

path = "/home/yuang/Desktop/3d_gaussian_splat/dataset/pre-trained_model/drjohnson/point_cloud/iteration_30000/point_cloud.ply"
max_sh_degree = 3

gm = GaussianModel(max_sh_degree)
gm.load_ply(path)

# quantize the features in each group separately, and save the quantized features to a ply file
bit_depth = 4
quantized_xyz = quantize_array(gm._xyz.detach().cpu().numpy(), bit_depth)
quantized_normal = quantize_array(gm._normal.detach().cpu().numpy(), bit_depth)
quantized_features_dc = quantize_array(gm._features_dc.detach().cpu().numpy(), bit_depth)
quantized_features_rest = quantize_array(gm._features_rest.detach().cpu().numpy(), bit_depth)
quantized_opacity = quantize_array(gm._opacity.detach().cpu().numpy(), bit_depth)
quantized_scaling = quantize_array(gm._scaling.detach().cpu().numpy(), bit_depth)
quantized_rotation = quantize_array(gm._rotation.detach().cpu().numpy(), bit_depth)

# save the quantized features to a ply file
quantize_group_list = ["position", "color", "size"]
for group in quantize_group_list:
    if group == "position":
        print("quantize position")
        gm.load_ply(path)
        gm._xyz = nn.Parameter(torch.tensor(quantized_xyz, dtype=torch.float, device="cuda").requires_grad_(True))
        gm.save_ply(f"/home/yuang/Desktop/3d_gaussian_splat/dataset/pre-trained_model/drjohnson/point_cloud/iteration_30000/point_cloud_{group}_quantized.ply")
    elif group == "color":
        print("quantize color")
        gm.load_ply(path)
        gm._normal = nn.Parameter(torch.tensor(quantized_normal, dtype=torch.float, device="cuda").requires_grad_(True))
        gm._features_dc = nn.Parameter(torch.tensor(quantized_features_dc, dtype=torch.float, device="cuda").requires_grad_(True))
        gm._features_rest = nn.Parameter(torch.tensor(quantized_features_rest, dtype=torch.float, device="cuda").requires_grad_(True))
        gm._opacity = nn.Parameter(torch.tensor(quantized_opacity, dtype=torch.float, device="cuda").requires_grad_(True))
        gm.save_ply(f"/home/yuang/Desktop/3d_gaussian_splat/dataset/pre-trained_model/drjohnson/point_cloud/iteration_30000/point_cloud_{group}_quantized.ply")
    elif group == "size":
        print("quantize size")
        gm.load_ply(path)
        gm._scaling = nn.Parameter(torch.tensor(quantized_scaling, dtype=torch.float, device="cuda").requires_grad_(True))
        gm._rotation = nn.Parameter(torch.tensor(quantized_rotation, dtype=torch.float, device="cuda").requires_grad_(True))
        gm.save_ply(f"/home/yuang/Desktop/3d_gaussian_splat/dataset/pre-trained_model/drjohnson/point_cloud/iteration_30000/point_cloud_{group}_quantized.ply")